In [19]:
import tensorflow as tf

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()


In [20]:
# Normalize the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0

# Flatten the images from 28x28 to a 1D array of 784 pixels
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))


In [21]:
# Normalize the pixel values to the range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0

# Flatten the images from 28x28 to a 1D array of 784 pixels
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))


In [22]:
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model


In [23]:
def find_learning_rate(model, X_train, y_train, loss_fn, lr_start=1e-4, lr_end=1.0, num_batches=100):
    lr_finder = tf.keras.experimental.CosineDecay(
        initial_learning_rate=lr_start,
        decay_steps=num_batches,
        alpha=lr_end / lr_start
    )
    optimizer = tf.keras.optimizers.Adam(lr_start)
    model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=1, batch_size=32, verbose=0)
    learning_rate = lr_finder(tf.range(num_batches))
    return learning_rate[-1].numpy()  # Return the final learning rate


In [24]:
model = build_model()
lr = find_learning_rate(model, X_train, y_train, 'sparse_categorical_crossentropy')
print("Found learning rate:", lr)


Found learning rate: 0.9997533


In [25]:
optimal_lr = 0.01  # Use the learning rate found from the learning rate finder
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_lr),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)


Epoch 1/20
1500/1500 [==============================] - 2s 943us/step - loss: 0.4447 - accuracy: 0.8572 - val_loss: 0.2213 - val_accuracy: 0.9304
Epoch 2/20
1500/1500 [==============================] - 1s 922us/step - loss: 0.2471 - accuracy: 0.9244 - val_loss: 0.1721 - val_accuracy: 0.9467
Epoch 3/20
1500/1500 [==============================] - 1s 904us/step - loss: 0.2011 - accuracy: 0.9384 - val_loss: 0.1382 - val_accuracy: 0.9571
Epoch 4/20
1500/1500 [==============================] - 1s 866us/step - loss: 0.1812 - accuracy: 0.9449 - val_loss: 0.1351 - val_accuracy: 0.9603
Epoch 5/20
1500/1500 [==============================] - 1s 873us/step - loss: 0.1632 - accuracy: 0.9509 - val_loss: 0.1242 - val_accuracy: 0.9641
Epoch 6/20
1500/1500 [==============================] - 1s 888us/step - loss: 0.1582 - accuracy: 0.9508 - val_loss: 0.1209 - val_accuracy: 0.9642
Epoch 7/20
1500/1500 [==============================] - 1s 868us/step - loss: 0.1474 - accuracy: 0.9548 - val_loss: 0.1138 -

In [26]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", test_accuracy)


313/313 [==============================] - 0s 383us/step - loss: 0.1084 - accuracy: 0.9708
Test accuracy: 0.97079998254776
